In [ ]:
import pymssql, re
from getpass import getpass
from time import sleep

###########################################
#  Automação para equalizar Servidores    #
###########################################

# login = input('Digite o Login (SQLServer):\n')
# senha = getpass('Digite a senha: ')

#  [destinos ]

servidores = []

# servidores = [ { 'ORIGEM' : [ 'DESTINOS' ] } ]

servidores = [{'servidorPRD1': ['DEV1','QA1','UAT1']}]
servidores += [{'servidorPRD2': ['DEV2','QA2','UAT2']}]

def query(pasta, arq):
    sql = ''
    for line in open('./{}/{}'.format(pasta,arq), 'r').readlines():
        sql += line
    return sql

def insert_many(cursor, dados, tabela, step = 1000):
    sql = " insert into " + tabela + " values "
    insert = []
    for r in range(0, len(dados), step):
        row = []
        insert = sql
        for e in dados[ r : r+step ]:
            values = []
            for n in e:
                values.append("'%s'" % str(n).replace("'", "''") if str(n) != 'None' else "NULL")
            row.append("({})".format(', '.join(values)))
        insert += ', '.join(row)
        cursor.execute(insert) 

def atualiza_destino(cursor):
    cursor.execute(query('creates', 'compara_estrutura.sql'))
    cursor.execute(query('creates', 'compara_index.sql'))
    cursor.execute(query('creates', 'compara_fks.sql'))

def remover_espacos_extras(texto):
    # Substitui qualquer quantidade de espaço em branco entre palavras por um único espaço
    texto_limpo = re.sub(r'\s+', ' ', texto)
    return texto_limpo.strip()

# Exemplo de uso
# texto = '''#=========== 
# #===
# create  
#           procedure
# ...
# '''
# texto_limpo = remover_espacos_extras(texto)
# print(texto_limpo)

def recreate_proc(destino):
    conn_destino = pymssql.connect(server=destino, database='tabelas_temp', autocommit=True)
    c_destino = conn_destino.cursor()
    sql = query('queries', 'recreate_procedures.sql')
    c_destino.execute(sql)
    create = c_destino.fetchall()
    base = ''
    for c in create:
        if(base != c[0]):
            conn_destino = pymssql.connect(server=destino, database=c[0], autocommit=True)
            c_destino = conn_destino.cursor()
        base = c[0]
        try:
            c_destino.execute(c[2]) # DROP 
            c_destino.execute(c[3]) # CREATE
        except:
            with open(f'procs_nao_executadas_{destino}_{c[0]}.sql', 'a+') as file:
                file.write('\n')
                file.write('use ' + c[0])
                file.write('\n')
                file.write(c[2])
                file.write('\n')
                file.write(c[3] if c[3] else '')

def execute_sql_destino(destino, c_destino, pasta, arq):
    print('Executando '+arq+' ...')
    sql = query(pasta, arq)
    c_destino.execute(sql)
    create = c_destino.fetchall()
    atualiza = False
    for c in create:
        try:
            c_destino.execute(c[0])
            atualiza = True
        except:
            with open(f'log_equalizar_servidores_{destino}.txt', 'a') as f:
                f.write(str(c)+'  --  falha ao executar \n')
    print('Executado')
    if(atualiza):
        print('Atualizando tabelas...')
        atualiza_destino(c_destino)


print('''
=========== Iniciando Atualização ===========
''')

for server in servidores:
    for origem in server.keys():
        for servers_destino in server.values():
            for destino in servers_destino:
                
                print(
                '''
                Atualizando o servidor         : {}
                Com base nos dados do servidor : {}
                '''.format(destino, origem)
                )

                conn_origem = pymssql.connect(server=origem,  database='tabelas_temp', autocommit=True)
                c_origem = conn_origem.cursor()

                conn_destino = pymssql.connect(server=destino,  database='tabelas_temp', autocommit=True)
                c_destino = conn_destino.cursor()

                print('Atualizando dados de origem...')
                sql = query('creates', 'origem_estrutura_index.sql')
                c_origem.execute(sql)
                
                # sql = query('creates', 'origem_procedures.sql')
                # c_origem.execute(sql)
                
                # sql = query('creates', 'origem_cdc.sql')
                # c_origem.execute(sql)

                print('Criando tabelas de dados no destino...')
                sql = query('creates', 'create_tables_destino.sql')
                c_destino.execute(sql)
                
                print('Atualizando tabelas de origem no destino...')
                atualiza_destino(c_destino)

                c_origem.execute('select * from tabelas_temp.dbo.t_bases_estrutura')
                estrutura = c_origem.fetchall()
                insert_many(c_destino, estrutura, 'tabelas_temp.dbo.t_bases_estrutura')
                del estrutura
                
                c_origem.execute('select * from tabelas_temp.dbo.t_bases_indexes')
                indexes = c_origem.fetchall()
                insert_many(c_destino, indexes, 'tabelas_temp.dbo.t_bases_indexes')
                del indexes
                
                # c_origem.execute('select * from tabelas_temp.dbo.t_bases_foreign_keys')
                # fks = c_origem.fetchall()
                # insert_many(c_destino, fks, 'tabelas_temp.dbo.t_bases_foreign_keys')
                # del fks

                # c_origem.execute('select * from tabelas_temp.dbo.t_bases_objects')
                # obj = c_origem.fetchall()
                # insert_many(c_destino, obj, 'tabelas_temp.dbo.t_bases_objects', 10)
                # del obj
                
                # c_origem.execute('select * from tabelas_temp.dbo.t_bases_cdc')
                # cdc = c_origem.fetchall()
                # insert_many(c_destino, cdc, 'tabelas_temp.dbo.t_bases_cdc')
                # del cdc


                # CREATE DATABASE
                # print('Criando Bases...')
                # execute_sql_destino(destino, c_destino, 'queries', 'create_database.sql')

                # # CREATE TABLES
                print('Criando Tabelas...')
                execute_sql_destino(destino, c_destino, 'queries', 'create_tables.sql')

                # ADD COLUMNS NOT EXISTS
                print('Criando Colunas...')
                execute_sql_destino(destino, c_destino, 'queries', 'add_columns_not_exists.sql')

                # ALTER COLUMNS TYPE
                print('Alterando Types...')
                execute_sql_destino(destino, c_destino, 'queries', 'alter_column_type.sql')

                # # DROP INDEXES
                # print('Dropando Indexes Desiguais...')
                # execute_sql_destino(destino, c_destino, 'queries', 'drop_indexes.sql')

                # # ADD INDEXES
                # print('Adicionando Indexes...')
                # execute_sql_destino(destino, c_destino, 'queries', 'add_indexes.sql')

                # ADD FOREIGN KEYS
                # print('Adicionando Foreign Keys...')
                # execute_sql_destino(destino, c_destino, 'queries', 'add_foreign_keys.sql')

                # CREATE CDC
                # print('Ajustando cdc...')
                # execute_sql_destino(destino, c_destino, 'queries', 'equalizar_cdc.sql')

                # RECREATE PROCEDURES
                # print('Recriando Procedures...')
                # recreate_proc(destino)

                print('''Servidor {} Atualizado'''.format(destino))

                

In [ ]:
import re

def remover_espacos_extras(texto):
    # Substitui qualquer quantidade de espaço em branco entre palavras por um único espaço
    texto_limpo = re.sub(r'\s+', ' ', texto)
    return texto_limpo.strip()

def substituir_case_insensitive(texto, velho, novo):
    # Usa expressão regular para substituir de forma case-insensitive
    texto_substituido = re.sub(re.escape(velho), novo, texto, flags=re.IGNORECASE)
    return texto_substituido

# Exemplo de uso
texto = '''#=========== 
#===
create  
          procedure
...
'''
texto_limpo = remover_espacos_extras(texto)
texto_final = substituir_case_insensitive(texto_limpo, 'CREATE procedure', 'alter procedure')
print(texto_final)


In [ ]:

if 'amazon' in 'Amazon CO':
    print(True)                